In [ ]:
<#
.SYNOPSIS
Formats a date string into various date formats.

.DESCRIPTION
This function takes a date string and formats it into one of the specified formats: ISO8601, RFC1123, Unix, or Custom. 
It also has an option to convert the date to UTC.

.PARAMETER dateString
The date string to be formatted. This parameter is mandatory.

.PARAMETER format
The format to convert the date string into. Valid values are "ISO8601", "RFC1123", "Unix", and "Custom". 
The default value is "ISO8601".

.PARAMETER UTC
A switch to indicate if the date should be converted to UTC. This parameter is optional.

.OUTPUTS
[string] The formatted date string.

.EXAMPLE
Format-Date -dateString "9/1/2021 2:12:16 PM" -format "RFC1123"
Returns: "Wed, 01 Sep 2021 14:12:16 GMT"

.EXAMPLE
Format-Date -dateString "9/1/2021 2:12:16 PM" -format "Unix"
Returns: 1630501936

.EXAMPLE
Format-Date -dateString "9/1/2021 2:12:16 PM" -format "Custom"
Returns: "2021-09-01 14:12:16"

.NOTES
Salesforce DateTime field values are stored and accessed in UTC format. 
When converting formatting the date, the UTC switch should only be used when the date is not already in UTC format.
The function will throw an error if the date string is not in a valid format. 

#>
function Format-DateTime {

    [CmdletBinding()]
    [OutputType([string])]
    param(
        [Parameter(Mandatory = $true, Position = 0)]
        [AllowEmptyString()]
        [Alias('d')]
        [string]$dateString,

        [Parameter(Mandatory = $false, Position = 1)]
        [ValidateSet("ISO8601", "RFC1123", "Unix", "Custom")]
        [Alias('f')]
        [string]$format = "ISO8601",

        [Parameter(Mandatory = $false, Position = 2)]
        [ValidateNotNullOrEmpty()]
        [Alias('custom')]
        [string]$customFormat,

        [Parameter(Mandatory = $false, Position = 3)]
        [switch] $UTC
    )

    if ($format -eq "Custom" -and -not $customFormat) {
        throw "Custom format string is required when format is set to 'Custom'."
    }

    if (-not $dateString) {
        return $null
    }

    try {
        $date = if ($UTC) { [DateTime]::Parse($dateString).ToUniversalTime() } else { [DateTime]::Parse($dateString) }

        switch ($format) {
            "ISO8601" {
                return $date.ToString("yyyy-MM-ddTHH:mm:ssZ")
            }
            "RFC1123" {
                return $date.ToString("R")
            }
            "Unix" {
                return [int]($date - (Get-Date "1970-01-01")).TotalSeconds
            }
            "Custom" {
                return $date.ToString($customFormat)
            }
        }
    }
    catch {
        throw "Invalid date string format, $dateString. Please provide a valid date string."
    }
    
}

<#
.SYNOPSIS
Converts a JSON string to a PowerShell object and extracts the 'Result' property. Primarily used for working with Salesforce API responses.

.DESCRIPTION
The Get-Result function takes a JSON string as input, converts it to a PowerShell object, and then selects and returns the 'Result' property from the converted object.

.PARAMETER json
The JSON string to be converted. This parameter is mandatory and accepts input from the pipeline.

.EXAMPLE
PS> '{"Result": {"Name": "John", "Age": 30}}' | Get-Result
Returns:
Name Age
---- ---
John 30
This example converts the JSON string and extracts the 'Result' property.

.INPUTS
System.String
The function accepts a JSON string as input.

.OUTPUTS
PSCustomObject
The function outputs the 'Result' property of the converted JSON object.

#>
function Get-Result {
    [CmdletBinding()]
    [OutputType([PSCustomObject])]
    param(
        [Parameter(Mandatory = $true, ValueFromPipeline = $true, Position = 0, HelpMessage = "Enter the JSON string to be converted.")]
        [Alias("j")]
        [string]$json
    )

    begin {
        $stringBuilder = New-Object System.Text.StringBuilder
    }

    process {
        [void]$stringBuilder.Append($json)
    }
    
    end {
        $response = $stringBuilder.ToString() | ConvertFrom-Json

        if ($response.status -eq 0) {
            return $response.result | Add-Member -MemberType NoteProperty -Name status -Value $response.status -PassThru
        }

        return [Object]@{
            status  = $response.status
            message = $response.message
        }
    }
}

<#
.SYNOPSIS
Retrieves a value from a dictionary based on a specified key and value. 
Gracefully handles finding a key from the hashtable with a potentially null value.
Assumes there is a $dictionary variable in the scope, which is a hashtable, but allows for a custom dictionary to be passed in.

.DESCRIPTION
The Get-DictionaryEntry function takes a key and a value as input parameters and retrieves the corresponding entry from a dictionary. If the value is not provided, the function returns null.

.PARAMETER key
The key to look up in the dictionary. This parameter is mandatory.

.PARAMETER value
The value to look up in the dictionary. This parameter is mandatory but allows empty strings.

.OUTPUTS
[string] The value associated with the specified key and value in the dictionary.

.EXAMPLE
PS> $dictionary = @{ "Name" = @{ "First" = "John"; "Last" = "Doe" } }
PS> Get-DictionaryEntry -key "Name" -value "First"
Returns: "John"

.EXAMPLE
PS> $dictionary = @{ "Name" = @{ "First" = "John"; "Last" = "Doe" } }
PS> Get-DictionaryEntry -key "Name" -value "Middle"
Returns: $null
#>
function Get-DictionaryEntry {
    [CmdletBinding()]
    [OutputType([string])]
    param(
        [Parameter(Mandatory = $true, Position = 0)]
        [ValidateNotNullOrEmpty()]
        [Alias('k')]
        [string]$key,

        [Parameter(Mandatory = $true, Position = 1)]
        [AllowEmptyString()]
        [Alias('v')]
        [string]$value,

        [Parameter(Mandatory = $false, Position = 2)]
        [ValidateNotNullOrEmpty()]
        [Alias('d')]
        [hashtable]$dict
    )

    if (-not $value) {
        return $null
    }

    if (-not $dict) {
        $dict = $dictionary
    }

    if (-not $dict.containskey($key)) {
        Write-Host "Key $key not found in dictionary."
        return $null
    }

    return $dict[$key][$value]
}

<#
.SYNOPSIS
Updates the binds dictionary with data from a specified JSON file.

.DESCRIPTION
The Update-Binds function takes a key, a file path, and an optional identifier as input parameters. It reads the JSON file, processes the data, and updates the binds dictionary with the corresponding entries. If the key is not found in the binds dictionary or the file path does not exist, the function throws an error.

.PARAMETER key
The key to look up in the binds dictionary. This parameter is mandatory.

.PARAMETER filePath
The path to the JSON file containing the data to update the binds dictionary. This parameter is mandatory.

.PARAMETER identifier
The identifier used to group the binds dictionary entries. The default value is "SourceId". This parameter is optional.

.OUTPUTS
None

.EXAMPLE
PS> Update-Binds -key "exampleKey" -filePath "C:\path\to\file.json"
Updates the binds dictionary with data from the specified JSON file using the key "exampleKey".

.EXAMPLE
PS> Update-Binds -key "exampleKey" -filePath "C:\path\to\file.json" -identifier "CustomId"
Updates the binds dictionary with data from the specified JSON file using the key "exampleKey" and the custom identifier "CustomId".

.NOTES
The function assumes that the binds dictionary is already defined in the scope. The JSON file should contain a 'status' property and a 'result' property with 'records.successfulResults' array.
#>
function Update-Binds {
    [CmdletBinding()]
    param(
        [Parameter(Mandatory = $true, Position = 0)]
        [ValidateNotNullOrEmpty()]
        [Alias('k')]
        [string]$key,

        [Parameter(Mandatory = $true, Position = 1)]
        [ValidateNotNullOrEmpty()]
        [Alias('f')]
        [string]$filePath,

        [Parameter(Mandatory = $false, Position = 2)]
        [Alias('i')]
        [string]$identifier = "SourceId"
    )

    if (-not $binds.containskey($key)) {
        throw "Key $key not found in binds."
    }

    if (!(Test-Path $filePath)) {
        throw "File path $filePath does not exist."
    }

    $result = Get-Content $filePath | ConvertFrom-Json
    if (-not $result.result.records -or $result.result.records.successfulResults.Count -eq 0) {
        throw "Error retrieving data from file: $($result.message)"
    }

    $bindMap = $binds[$key] | Group-Object -Property $identifier -AsHashTable -AsString

    $result = $result.result
    $result.records.successfulResults | ForEach-Object {
        $bind = $bindMap[$_.Legacy_Id__c]

        if (-not $bind) {
            return
        }

        $bind | Add-Member -MemberType NoteProperty -Name TargetId -Value $_.sf__Id -PassThru -Force | Out-Null
    }

}

<#
.SYNOPSIS
Splits an array into smaller arrays (slices) of a specified size.

.DESCRIPTION
The New-ArraySlices function takes an array of items and splits it into smaller arrays (slices) of a specified size. This can be useful for processing large arrays in smaller chunks.

.PARAMETER Item
The array of items to be split into slices. This parameter is mandatory and accepts input from the pipeline.

.PARAMETER Size
The size of each slice. The default value is 10.

.OUTPUTS
[array[]] An array of arrays, where each inner array is a slice of the original array.

.EXAMPLE
PS> 1..25 | New-ArraySlices -Size 5
Returns:
1  2  3  4  5
6  7  8  9  10
11 12 13 14 15
16 17 18 19 20
21 22 23 24 25

.EXAMPLE
PS> $data = "a","b","c","d","e","f","g","h","i","j","k"
PS> New-ArraySlices -Item $data -Size 3
Returns:
a b c
d e f
g h i
j k

.NOTES
This function is useful for processing large arrays in smaller chunks to avoid memory issues or to parallelize processing.
#>
function New-ArraySlices {
    [CmdletBinding()]
    param (
        [Parameter(Mandatory = $true, Position = 0, ValueFromPipeline = $true)]
        [ValidateNotNullOrEmpty()]
        [Alias('a')]
        [PSObject[]]$array,

        [Parameter(Mandatory = $false, Position = 1)]
        [ValidateRange(1, 1000)]
        [Alias('s')]
        [int] $size = 10
    )
    BEGIN { 
        $items = [System.Collections.Generic.List[PSObject]]::new()
        $slices = [System.Collections.Generic.List[PSObject[]]]::new()
    }
    PROCESS {
        $array | ForEach-Object { $items.Add($_) }
    }
    END {
        0..($items.Count - 1) | ForEach-Object -Process {
            if ($_ % $size -eq 0) {
                $slice = $items.GetRange($_, [math]::Min($size, $items.Count - $_))
                $slices.Add($slice)
            }
        }
        $slices
    }
}

In [ ]:
Format-DateTime -d "2002-01-01" -f "ISO8601" 

### Set the target and source org variables. The target is the org that the data will be copied to and the source is the org that the data will be copied from.

In [ ]:
#Set target and source orgs:
$target = "TCSValidate"; $source = "TCSErx";

#Set the Institution
$institution = 'Pacific Oaks College'

# Check to see if already in the directory
if ((Get-Location) -contains $institution) {
    Write-Host "Already in the directory"
    return
}

#Create a directory for the institution if doesn't exist
if (!(Test-Path -Path $institution)) {
    New-Item -ItemType Directory -Name $institution
}

"data", "results", "logs" | ForEach-Object {
    $path = Join-Path -Path $institution -ChildPath $_
    if (!(Test-Path -Path $path)) {
        New-Item -ItemType Directory -Name $path
    }
}

#Set the default directory
Set-Location $institution

In [ ]:
# Set the flag to indicate if records should be inserted.
$validateOnly = $false

### Import the dictionary of translated values for fields that are mapped to enumerated picklist values. Also import bindings the bindings for the PK from each org.

In [ ]:
$dictionary = Get-Content -Path ..\dictionary.json | ConvertFrom-Json -AsHashTable

$binds = @{}

### Get the list of users from the source org and map them by their email address. Then query the target org and match the users by email address.

In [ ]:
$binds["User"] = [System.Collections.Generic.List[PSCustomObject]]::new()

sf data query -q "SELECT Id, Username, Email FROM User WHERE UserType = 'Standard'" -o $source --json |
Get-Result |
Select-Object -ExpandProperty Records |
ForEach-Object {
    $binds["User"].Add(
        [PSCustomObject]@{
            SourceId    = $_.Id
            ReferenceId = $_.Email.toLower()
        }
    )
}


sf data query -q "SELECT Id, Username, Email FROM User WHERE UserType = 'Standard' AND isActive = TRUE" -o TCSProd --json |
Get-Result |
Select-Object -ExpandProperty records |
ForEach-Object {
    $key = $_.Email.ToLower()

    $bind = $binds["User"] | Where-Object { $_.ReferenceId -eq $key }

    if (-not $bind) {
        return
    }

    if ($bind.TargetId) {
        "[$(Get-Date)] Duplicate Email Address Found for User: $($bind.ReferenceId) in $target" | Out-File -Append -FilePath ..\logs\Users.log
        return
    }

    $bind | Add-Member -MemberType NoteProperty -Name "TargetId" -Value $_.Id -PassThru
} | Out-Null

$binds["User"] = $binds["User"] | Where-Object { $null -ne $_.TargetId }

### Create a hashset of US States and Provinces for filling in the blanks on Country Codes.

In [ ]:
# Create a HashSet containing the two-letter abbreviation of each US state or territory
$USStates = @("AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY", "DC", "AS", "GU", "MP", "PR", "VI", "UM", "FM", "MH", "PW")
$CanadianProvinces = @("AB", "BC", "MB", "NB", "NL", "NS", "NT", "NU", "ON", "PE", "QC", "SK", "YT")

Function Get-CountryCode {
    param(
        [Parameter(Mandatory = $true, Position = 0)]
        [AllowEmptyString()]
        [string]$state
    )

    $state = $state.ToUpper()

    if ($USStates.Contains($state)) {
        return "US"
    }
    elseif ($CanadianProvinces.Contains($state)) {
        return "CA"
    } 
    # If the state is not found in either list, return null
    return $null
}

# Start of Reference Data Migration

## Reference Data Objects
  1. Institution (Account)
  2. Campus (Location)
  3. Program (Learning Program)
  4. Terms (Academic Term)
  5. Program + Term + Campus union (PTAT)
  6. Scholarships (Scholarship)

In [ ]:
# Fetch the RecordTypeId for the College Accounts
$collegeAccountRT = sf data query -q "SELECT Id FROM RecordType WHERE SObjectType = 'Account' AND DeveloperName = 'College'" -o $source --json |
ConvertFrom-Json |
Select-Object -ExpandProperty Result |
Select-Object -ExpandProperty Records |
Select-Object -ExpandProperty Id

# Add the binds for Accounts
$binds["Account"] = [System.Collections.Generic.List[PSCustomObject]]::new()


$fields = @(
    "Id", 
    "Name", 
    "EnrollmentrxRx__School_Address__c", 
    "EnrollmentrxRx__School_City__c", 
    "EnrollmentrxRx__School_State_Province__c",
    "EnrollmentrxRx__School_Zip__c",
    "CVUE_ID__c",
    "CreatedDate",
    "LastModifiedDate"
)
    
$records = sf data query -q "SELECT $($fields -join ",") FROM EnrollmentrxRx__School__c" -o $source --json | 
ConvertFrom-Json |
Select-Object -ExpandProperty result |
Select-Object -ExpandProperty records |
ForEach-Object {
    $binds["Account"].add([PSCustomObject]@{
            SourceId = $_.Id
        })

    $billingState = Get-DictionaryEntry -k "states" -v $_.EnrollmentrxRx__School_State_Province__c

    return [PSCustomObject]@{
        Name               = $_.Name
        Type               = "Partner"
        RecordTypeId       = $collegeAccountRT
        BillingStreet      = $_.EnrollmentrxRx__School_Address__c
        BillingCity        = $_.EnrollmentrxRx__School_City__c
        BillingStateCode   = $billingState
        BillingPostalCode  = $_.EnrollmentrxRx__School_Zip__c
        BillingCountryCode = if ($billingState) { "US" } else { $null }
        Anthology_ID__c    = $_.CVUE_ID__c
        Legacy_Id__c       = $_.Id
        CreatedDate        = Format-DateTime -d $_.CreatedDate
        LastModifiedDate   = Format-DateTime -d $_.LastModifiedDate
    }
} | Export-Csv -Path data\InstitutionAccounts.csv -NoTypeInformation -Encoding UTF8


Write-Host ($records | ConvertTo-Json -Depth 2)
Remove-Variable -Name collegeAccountRT

In [ ]:
if ($validateOnly) {
    Write-Host "Validation only. No records will be inserted."
}

# See https://developer.salesforce.com/docs/atlas.en-us.sfdx_cli_reference.meta/sfdx_cli_reference/cli_reference_data_commands_unified.htm#cli_reference_data_upsert_bulk_unified
sf data upsert bulk --sobject Account --file data\InstitutionAccounts.csv --external-id Legacy_Id__c --target-org $target --json --wait 10 | Out-File -FilePath results\Account.json

# Update the binds with the target IDs in the event of a successful upsert.
Update-Binds -key "Account" -filePath results\Account.json

In [ ]:
$binds["Location"] = [System.Collections.Generic.List[PSCustomObject]]::new()
$binds["Address"] = [System.Collections.Generic.List[PSCustomObject]]::new()

$fields = @(
    "Id",
    "Name",
    "CVUE_ID__c"
    "EnrollmentrxRx__Address_of_Campus__c",
    "EnrollmentrxRx__Campus_City__c",
    "EnrollmentrxRx__Campus_State_Province__c",
    "EnrollmentrxRx__Campus_Zip__c"
    "School__c",
    "ShiftID__c",
    "Sparkroom_ID__c",
    "Status__c",
    "X800_Number__c",
    "CreatedDate",
    "LastModifiedDate"
)

$predicate = "EnrollmentrxRx__Campus__c.School__r.Name = '$institution'"


$campuses = sf data query -q "SELECT $($fields -join ",") FROM EnrollmentrxRx__Campus__c WHERE $predicate" --target-org $source --json | 
Get-Result |
Select-Object -ExpandProperty Records |
ForEach-Object {
    $binds["Location"].Add([PSCustomObject]@{
            SourceId = $_.Id
        })
    
    $billingState = Get-DictionaryEntry -key "states" -value $_.EnrollmentrxRx__Campus_State_Province__c

    $address = $null
    if ($_.EnrollmentrxRx__Address_of_Campus__c -or $_.EnrollmentrxRx__Campus_City__c -or $billingState -or $_.EnrollmentrxRx__Campus_Zip__c) {
        $binds["Address"].Add([PSCustomObject]@{
                SourceId = $_.Id
            })
        
        $address = [PSCustomObject]@{
            Street                  = $_.EnrollmentrxRx__Address_of_Campus__c
            City                    = $_.EnrollmentrxRx__Campus_City__c
            StateCode               = $billingState
            PostalCode              = $_.EnrollmentrxRx__Campus_Zip__c
            CountryCode             = if ($billingState) { "US" } else { $null }
            LocationType            = "Campus"
            "ParentId.Legacy_Id__c" = $_.Id
            CreatedDate             = Format-DateTime -d $_.CreatedDate
            LastModifiedDate        = Format-DateTime -d $_.LastModifiedDate
        }
    }

    
    $location = [PSCustomObject]@{
        Name                       = $_.Name
        LocationType               = "Campus"
        Legacy_Id__c               = $_.Id
        "Provider__r.Legacy_Id__c" = $_.School__c
        Sparkroom_ID__c            = $_.Sparkroom_ID__c
    }
    
    return @{
        "Location" = $location
        "Address"  = $address
    }
}

$campuses | Select-Object -ExpandProperty Location | Export-Csv -Path data\Locations.csv -NoTypeInformation -Encoding UTF8
$campuses | Where-Object { $_.Address } | Select-Object -ExpandProperty Address | Export-Csv -Path data\Addresses.csv -NoTypeInformation -Encoding UTF8

Remove-Variable -Name campuses

In [ ]:
if ($validateOnly) {
    Write-Host "Validation only. No records will be inserted."
    return
}

sf data upsert bulk --sobject Location --file data\Locations.csv --external-id Legacy_Id__c --target-org $target --json --wait 10 | Out-File -FilePath results\Location.json
sf data upsert bulk --sobject Address --file data\Addresses.csv --external-id Id --target-org $target --json --wait 10 | Out-File -FilePath results\Address.json

# Update the binds with the target IDs in the event of a successful upsert.
Update-Binds -key "Location" -filePath results\Location.json

In [ ]:
$binds["Learning"] = [System.Collections.Generic.List[PSCustomObject]]::new()
$binds["LearningProgram"] = [System.Collections.Generic.List[PSCustomObject]]::new()

$fields = @(
    "Id",
    "Name",
    "EnrollmentrxRx__Description__c",
    "EnrollmentrxRx__School_Offering_Program__c",
    "ProgramLevel__c",
    "SparkroomProgramId__c",
    "Status__c",
    "CreatedDate",
    "LastModifiedDate"
)

$predicate = "EnrollmentrxRx__School_Offering_Program__r.Name = '$institution'"

$learnings = sf data query -q "SELECT $($fields -join ",") FROM EnrollmentrxRx__Program_Catalog__c WHERE $predicate" -o $source --json |
Get-Result |
Select-Object -ExpandProperty Records |
ForEach-Object {
    $binds["Learning"].Add([PSCustomObject]@{
            SourceId = $_.Id
        })
    
    $binds["LearningProgram"].Add([PSCustomObject]@{
            SourceId = $_.Id
        })
    
    $learning = [PSCustomObject]@{
        Name                    = $_.Name
        Description             = $_.EnrollmentrxRx__Description__c
        Legacy_Id__c            = $_.Id
        AcademicLevel           = Get-DictionaryEntry -k "academicLevels" -v $_.ProgramLevel__c
        IsActive                = $_.Status__c -eq "Active"
        CreatedDate             = Format-DateTime -d $_.CreatedDate
        LastModifiedDate        = Format-DateTime -d $_.LastModifiedDate
        "Provider.Legacy_Id__c" = $_.EnrollmentrxRx__School_Offering_Program__c
    }

    $learningProgram = [PSCustomObject]@{
        Name                    = $_.Name
        Description             = $_.EnrollmentrxRx__Description__c
        Legacy_Id__c            = $_.Id
        "Learning.Legacy_Id__c" = $_.Id
        Deposit_Amount__c       = if ($_.Deposit_Amount_Override__c) { $_.Deposit_Amount_Override__c } else { $_.Default_Deposit_Amount__c }
        Application_Fee__c      = if ($_.Application_Fee_Override__c) { $_.Application_Fee_Override__c } else { $_.Application_Fee__c }
        Sparkroom_Id__c         = $_.SparkroomProgramId__c
        #Licensure_Approved_States__c = ($_."Licensure_Approved_States__c" -split ',\s+') -join ";"
        #No_Recruit_States__c         = ($_.No_Recruit_State__c -split ',\s+') -join ";" 
        CreatedDate             = Format-DateTime -d $_.CreatedDate
        LastModifiedDate        = Format-DateTime -d $_.LastModifiedDate
    }
    
    return @{
        "Learning"        = $learning
        "LearningProgram" = $learningProgram
    }
}

$learnings | Select-Object -ExpandProperty Learning | Export-Csv -Path data\Learning.csv -NoTypeInformation -Encoding UTF8
$learnings | Select-Object -ExpandProperty LearningProgram | Export-Csv -Path data\LearningProgram.csv -NoTypeInformation -Encoding UTF8

In [ ]:
if ($validateOnly) {
    Write-Host "Validation only. No records will be inserted."
    return
}

sf data upsert bulk --sobject Learning --file data\Learning.csv --external-id Legacy_Id__c --target-org $target --json --wait 10 | Out-File -FilePath results\Learning.json
sf data upsert bulk --sobject LearningProgram --file data\LearningProgram.csv --external-id Legacy_Id__c --target-org $target --json --wait 10 | Out-File -FilePath results\LearningProgram.json

# Update the binds with the target IDs in the event of a successful upsert.
Update-Binds -key "Learning" -filePath results\Learning.json 
Update-Binds -key "LearningProgram" -filePath results\LearningProgram.json

In [ ]:
$binds["AcademicTerm"] = [System.Collections.Generic.List[PSCustomObject]]::new()

$fields = @(
    "Id",
    "Name",
    "CampusVueTermId__c",
    "Associated_Institution__c",
    "EnrollmentrxRx__Add_Drop_Date__c",
    "EnrollmentrxRx__Last_Day_To_Register__c",
    "EnrollmentrxRx__Term_Desctiption__c",
    "EnrollmentrxRx__Term_End_Date__c",
    "EnrollmentrxRx__Term_Start_Date__c",
    "Term_Status__c",
    "TermCode__c",
    "CreatedDate",
    "LastModifiedDate"
)


sf data query -q "SELECT $($fields -join ",") FROM EnrollmentrxRx__Term__c WHERE Associated_Institution__r.Name = '$institution'" --target-org $source --json | 
Get-Result |
Select-Object -ExpandProperty Records |
ForEach-Object {
    $binds["AcademicTerm"].Add([PSCustomObject]@{
            SourceId = $_.Id
        })

    [PSCustomObject]@{
        Name                          = $_.Name
        Legacy_Id__c                  = $_.Id
        Description                   = $_.EnrollmentrxRx__Term_Desctiption__c
        StartDate                     = Format-DateTime -d $_.EnrollmentrxRx__Term_Start_Date__c 
        EndDate                       = Format-DateTime -d $_.EnrollmentrxRx__Term_End_Date__c
        Census_Date__c                = Format-DateTime -d $_.EnrollmentrxRx__Add_Drop_Date__c
        RegistrationCloseDate         = Format-DateTime -d $_.EnrollmentrxRx__Last_Day_To_Register__c
        SIS_TermID__c                 = $_.CampusVueTermId__c
        SIS_Term_Code__c              = $_.TermCode__c
        IsActive                      = $_.Term_Status__c -eq "Active"
        "Institution__r.Legacy_Id__c" = $_.Associated_Institution__c
        CreatedDate                   = Format-DateTime -d $_.CreatedDate
        LastModifiedDate              = Format-DateTime -d $_.LastModifiedDate
    }
} | Export-Csv -Path data\AcademicTerm.csv -NoTypeInformation -Encoding UTF8

In [ ]:
if ($validateOnly) {
    Write-Host "Validation only. No records will be inserted."
    return
}

sf data upsert bulk --sobject AcademicTerm --file data\AcademicTerm.csv --external-id Legacy_Id__c --target-org $target --json --wait 10 | Out-File -FilePath results\AcademicTerm.json

# Update the binds with the target IDs in the event of a successful upsert.
Update-Binds -key "AcademicTerm" -filePath results\AcademicTerm.json

In [ ]:
$binds["ProgramTermApplnTimeline"] = [System.Collections.Generic.List[PSCustomObject]]::new()

$fields = @(
    "Id", 
    "Program_Offered__c", 
    "Program_Offered__r.EnrollmentrxRx__Program_Catalog__c", 
    "Program_Offered__r.EnrollmentrxRx__Campus_Offering_this_Program__c", 
    "Term__c",
    "CreatedDate",
    "LastModifiedDate"
)

$predicate = "WHERE Term__r.EnrollmentrxRx__Term_Start_Date__c >= 2021-05-10 AND Term__r.Associated_Institution__r.Name IN ('$institution')"

sf data query -q "SELECT $($fields -join ",") FROM Program_Term__c $predicate" -o $source --json |
Get-Result | 
Select-Object -ExpandProperty Records | 
ForEach-Object {
    $program = $_.Program_Offered__r.EnrollmentrxRx__Program_Catalog__c
    $term = $_.Term__c
    $campus = $_.Program_Offered__r.EnrollmentrxRx__Campus_Offering_this_Program__c
    
    $compositeKey = "$($program)-$($term)-$($campus)"

    $binding = $binds["ProgramTermApplnTimeline"] | Where-Object { $_.CompositeId -eq $compositeKey }
    if ($binding) {
        return
    }

    $binds["ProgramTermApplnTimeline"].Add([PSCustomObject]@{
            SourceId    = $_.Id
            CompositeId = $compositeKey
        })
    

    [PSCustomObject]@{
        Legacy_Id__c                   = $_.Id
        "LearningProgram.Legacy_Id__c" = $_.Program_Offered__r.EnrollmentrxRx__Program_Catalog__c
        "Location__r.Legacy_Id__c"     = $campus
        "AcademicTerm.Legacy_Id__c"    = $term
        CreatedDate                    = Format-DateTime -d $_.CreatedDate
        LastModifiedDate               = Format-DateTime -d $_.LastModifiedDate
    }
} | Export-Csv -Path data\ProgramTermApplnTimeline.csv -NoTypeInformation -Encoding UTF8

In [ ]:
if ($validateOnly) {
    Write-Host "Validation only. No records will be inserted."
    return
}

sf data upsert bulk --sobject ProgramTermApplnTimeline --file data\ProgramTermApplnTimeline.csv --external-id Legacy_Id__c --target-org $target --json --wait 10 | Out-File -FilePath results\ProgramTermApplnTimeline.json

# Update the binds with the target IDs in the event of a successful upsert.
Update-Binds -key "ProgramTermApplnTimeline" -filePath results\ProgramTermApplnTimeline.json

In [ ]:
$binds["Scholarship"] = [System.Collections.Generic.List[PSCustomObject]]::new()

$fields = @(
    "Id",
    "Location__c",
    "Scholarship__c",
    "Scholarship__r.Name",
    "Scholarship__r.AllowedAwardAmounts__c",
    "Scholarship__r.Amount__c",
    "Scholarship__r.AwardCriteria1Options__c",
    "Scholarship__r.AwardCriteria2Options__c",
    "Scholarship__r.AwardCriteria1Required__c",
    "Scholarship__r.AwardCriteria2Required__c",
    "Scholarship__r.Scholarship_Status__c",
    "Scholarship__r.Display_Name__c",
    "Scholarship__r.Institution__c",
    "Scholarship__r.CreatedDate",
    "Scholarship__r.LastModifiedDate"
)

$predicate = "WHERE Scholarship__r.Institution__r.Name = 'Institution'"

sf data query -q "SELECT $($fields -join ",") FROM Scholarship_Location__c $predicate" -o $source --json |
Get-Result |
Select-Object -ExpandProperty Records |
ForEach-Object {
    $binds["Scholarship"].Add([PSCustomObject]@{
            SourceId = $_.Id
        })

    $scholarship = $_.Scholarship__r

    [PSCustomObject]@{
        Name                          = $scholarship.Name
        Legacy_Id__c                  = $_.Id
        Allowed_Award_Amounts__c      = $scholarship.AllowedAwardAmounts__c
        Award_Criteria_1_Options__c   = $scholarship.AwardCriteria1Options__c
        Award_Criteria_1_Required__c  = [bool]$scholarship.AwardCriteria1Required__c
        Award_Criteria_2_Options__c   = $scholarship.AwardCriteria2Options__c
        Award_Criteria_2_Required__c  = [bool]$scholarship.AwardCriteria2Required__c
        Active__c                     = $scholarship.Scholarship_Status__c -eq "Active"
        Public_Label__c               = $scholarship.Display_Name__c
        "Institution__r.Legacy_Id__c" = $scholarship.Institution__c
        "Location__r.Legacy_Id__c"    = $_.Location__c
        CreatedDate                   = Format-DateTime -d $scholarship.CreatedDate
        LastModifiedDate              = Format-DateTime -d $scholarship.LastModifiedDate
    }
} | Export-Csv -Path data\Scholarship.csv -NoTypeInformation -Encoding UTF8


In [ ]:
if ($validateOnly) {
    Write-Host "Validation only. No records will be inserted."
    return
}

sf data upsert bulk --sobject Scholarship__c --file data\Scholarship.csv --external-id Legacy_Id__c --target-org $target --json --wait 10 | Out-File -FilePath results\Scholarship.json

# Update the binds with the target IDs in the event of a successful upsert.
Update-Binds -key "Scholarship" -filePath results\Scholarship.json

In [ ]:
# Cache the bindings 
$binds | ConvertTo-Json -Depth 2 | Out-File -FilePath bindings.json

# Start of Student Data Migration

## Student Data Objects
  1. Contact (Person Account)
  2. Enrollment Opportunity (Opportunity and Individual Application)
  3. Activity (Task and Event)
  4. Award (Scholarship Award)

In [ ]:
# Optional restore binds in the event of a split data load

$b = @{} 
$content = Get-Content -Path bindings.json -Raw | 
ConvertFrom-Json

$content | Get-Member -MemberType NoteProperty | # Iterate over each property and add as a key to a hash map
ForEach-Object {

    $b.$($_.Name) = [System.Collections.Generic.List[PSObject]]$content.$($_.Name) #Likewise, each property value must also be converted to a hash map from an object with keys
}

$binds = $b

Remove-Variable -Name b

In [ ]:
$users = $binds["User"] | Group-Object -Property SourceId -AsHashTable
# Set a default owner for records where the original owner is not an active user in the target instance.
# Change the name predicate to match the desired default owner.
$defaultOwnerId = sf data query -q "SELECT Id FROM User WHERE Name = 'Tondro Consulting' AND isActive = TRUE AND UserType = 'Standard'" -o $target --json |
Get-Result |
Select-Object -ExpandProperty Records |
Select-Object -First 1 |
Select-Object -ExpandProperty Id

In [ ]:
$binds["PersonAccount"] = [System.Collections.Generic.List[PSCustomObject]]::new()
#$binds["Individual"] = [System.Collections.Generic.List[PSCustomObject]]::new()

#$individuals = [System.Collections.Generic.List[PSCustomObject]]::new()

# Fetch the RecordTypeId for the College Accounts
$personAccountRT = sf data query -q "SELECT Id FROM RecordType WHERE SObjectType = 'Account' AND DeveloperName = 'PersonAccount'" -o $target --json |
ConvertFrom-Json |
Select-Object -ExpandProperty Result |
Select-Object -ExpandProperty Records |
Select-Object -ExpandProperty Id

$fields = @(
    "Id",
    'Are_you_Hispanic_or_Latino__c',
    'Associated_Institution__c',
    'Best_Method_of_Contact_1st_Choice__c',
    'Best_Method_of_Contact_2nd_Choice__c',
    'Best_Time_To_Contact__c',
    'Birth_City__c',
    'Birth_Country__c',
    'Birth_State__c',
    'Birthdate',
    'Citizenship__c',
    'Do_Not_Call_Home__c',
    'Do_Not_Call_Mobile__c',
    'Do_Not_Call_Other__c',
    'Do_Not_Call_Work__c',
    'Email',
    'Email__c',
    'English_Primary_Language__c',
    'EnrollmentrxRx__Secondary_Email__c',
    'EnrollmentrxRx__SIS_ID__c',
    'Ethnicity__c',
    'FirstName',
    'Former_Last_Name__c',
    'Gender_Identity__c',
    'Has_Opted_In_for_Text_Messaging__c',
    'HasOptedOutOfEmail',
    'HomePhone',
    'LastName',
    'Level_of_Education__c',
    'MailingCity',
    'MailingCountry',
    'MailingPostalCode',
    'MailingState',
    'MailingStreet',
    'Middle_Name__c',
    'Military_Status__c',
    'MobilePhone',
    'OtherCity',
    'OtherCountry',
    'OtherPostalCode',
    'OtherState',
    'OtherStreet',
    'Phone',
    'Pronoun__c',
    'Salutation',
    'Sex_Legal__c',
    'Sexual_Orientation__c',
    'SSN__c',
    'Work_Phone__c',
    'CreatedDate',
    'LastModifiedDate',
    'OwnerId'
)

$predicate = "WHERE Id IN (SELECT EnrollmentrxRx__Applicant__c FROM EnrollmentrxRx__Enrollment_Opportunity__c WHERE (Program_Term__r.Term__r.EnrollmentrxRx__Term_Start_Date__c >= 2021-05-10 AND Program_Term__r.Term__r.Associated_Institution__r.Name IN ('$institution')) OR (CreatedDate >= 2022-06-01T00:00:00z  AND CreatedDate <= 2025-02-07T00:00:00z AND Program_Term__c = NULL AND School__r.Name IN ('$institution')))"

sf data query -q "SELECT $($fields -join ",") FROM Contact $predicate" -o $source --json |
Get-Result | 
Select-Object -ExpandProperty Records |
ForEach-Object {
    
    $binds["PersonAccount"].Add([PSCustomObject]@{
            SourceId = $_.Id
        })

    $owner = if ($users.containsKey($_.OwnerId)) { $users[$_.OwnerId].TargetId } else { $defaultOwnerId }

    [PSCustomObject]@{
        RecordTypeId                    = $personAccountRT
        Legacy_Id__c                    = $_.Id
        Legacy_Id__pc                   = $_.Id
        FirstName                       = $_.FirstName
        LastName                        = $_.LastName
        Former_Last_Name__pc            = $_.Former_Last_Name__c
        PersonBirthdate                 = $_.Birthdate
        Ethnicity__pc                   = $_.Are_you_Hispanic_or_Latino__c
        School__c                       = $institution
        Preferred_Contact_1__c          = $_.Best_Method_of_Contact_1st_Choice__c
        Preferred_Contact_2__c          = $_.Best_Method_of_Contact_2nd_Choice__c
        Birth_City__pc                  = $_.Birth_City__c
        Birth_Country__pc               = $_.Birth_Country__c
        Birth_State__pc                 = $_.Birth_State__c
        Citizenship_Status__pc          = if ($_.Citizenship__c -eq 'INT') { 'International' } else { $_.Citizenship__c }
        PersonEmail                     = $_.Email__c
        Is_English_Primary_Language__pc = $_.English_Primary_Language__c
        Alternate_Email__pc             = $_.EnrollmentrxRx__Secondary_Email__c
        SyStudentID__pc                 = $_.EnrollmentrxRx__SIS_ID
        Race__pc                        = $_.Ethnicity__c
        PersonGenderIdentity            = $_.Gender_Identity__c -replace "non-conforming", "non-confirming" #The Picklist value for this has "non-confirming" instead of "non-conforming" all other values match 1:1
        Mogli_SMS__Mogli_Opt_Out__pc    = !([bool]$_.Has_Opted_In_for_Text_Messaging__c)
        PersonHasOptedOutOfEmail        = $_.HasOptedOutOfEmail
        PersonHomePhone                 = $_.HomePhone
        PersonLeadSource                = $_.LeadSource
        PersonMailingStreet             = $_.MailingStreet -replace '\n', ' '
        PersonMailingCity               = $_.MailingCity
        PersonMailingStateCode          = Get-DictionaryEntry -k "states" -v $_.MailingState
        PersonMailingPostalCode         = $_.MailingPostalCode
        PersonMailingCountryCode        = Get-CountryCode (Get-DictionaryEntry -k "states" -v $_.MailingState)
        MiddleName                      = $_.Middle_Name__c
        Military_Status__pc             = $militaryStatus
        PersonMobilePhone               = $_.MobilePhone
        PersonOtherStreet               = $_.OtherStreet -replace '\n', ' '
        PersonOtherCity                 = $_.OtherCity
        PersonOtherStateCode            = Get-DictionaryEntry -k "states" -v $_.OtherState
        PersonOtherPostalCode           = $_.OtherPostalCode
        PersonOtherCountryCode          = Get-CountryCode (Get-DictionaryEntry -k "states" -v $_.OtherState)
        Phone                           = $_.Phone
        Salutation                      = $_.Salutation
        PersonPronouns                  = Get-DictionaryEntry -k "pronouns" -v $_.Pronoun__c
        Sex__pc                         = $_.Sex_Legal__c
        Sexual_Orientation__pc          = $_.Sexual_Orientation__c
        Social_Security_Number__pc      = $_.SSN__c
        Work_Phone__pc                  = $_.Work_Phone__c
        CreatedDate                     = Format-DateTime -d $_.CreatedDate
        LastModifiedDate                = Format-DateTime -d $_.LastModifiedDate
        OwnerId                         = $owner
    }
} | Export-Csv -Path data\PersonAccount.csv -NoTypeInformation -Encoding UTF8

In [ ]:
if ($validateOnly) {
    Write-Host "Validation only. No records will be inserted."
    return
}

sf data upsert bulk --sobject Account --file data\PersonAccount.csv --external-id Legacy_Id__c --target-org $target --json --wait 10 | Out-File -FilePath results\PersonAccount.json

# Update the binds with the target IDs in the event of a successful upsert.
Update-Binds -key "PersonAccount" -filePath results\PersonAccount.json

In [ ]:
#Query the Accounts and get the PersonContactIds

New-ArraySlices -array $binds["PersonAccount"] -size 750 | ForEach-Object {
    $group = $_ | Where-Object { $_.TargetId } | Group-Object -Property TargetId -AsHashTable

    sf data query -q "SELECT Id, PersonContactId FROM Account WHERE Id IN ('$($group.Keys -join "','")')" -o $target --json |
    Get-Result |
    Select-Object -ExpandProperty Records |
    ForEach-Object {
        $bind = $group[$_.Id]
        $bind | Add-Member -MemberType NoteProperty -Name "PersonContactId" -Value $_.PersonContactId -PassThru | Out-Null
    }
}

In [ ]:
$binds | ConvertTo-Json -Depth 2 | Out-File -FilePath bindings.json

### Opportunity and Individual Application special behavior
The Opportunity and Individual Application have relationships that may or may not be set, depending on if the term, program, and location were set on the source record.
Setting a relationship field using the relationship record's external ID is not supported with a nil/null value.
As a workaround, External ID fields are not used on any nillable fields. 
Instead, the potential related records are set by taking the bindings for the respective object and stored into a hashmap, using the source ID as the key. 

Potential nillable fields include:
- Program
- Term
- Campus
- ProgramTermApplnTimeline

In [ ]:
$personAccounts = sf data query -q "SELECT Id, FirstName, LastName, PersonContactId, Legacy_Id__c FROM Account WHERE IsPersonAccount = TRUE AND Legacy_Id__c != NULL AND School__c = '$institution'" -o $target --json |
Get-Result |
Select-Object -ExpandProperty Records |
Group-Object -Property Legacy_Id__c -AsHashTable

$programs = sf data query -q "SELECT Id, Name, Legacy_Id__c FROM LearningProgram WHERE Legacy_Id__c != NULL AND Learning.Provider.Name = '$institution'" -o $target --json |
Get-Result |
Select-Object -ExpandProperty Records |
Group-Object -Property Legacy_Id__c -AsHashTable


$terms = $binds["AcademicTerm"] | Group-Object -Property SourceId -AsHashTable
$locations = $binds["Location"] | Group-Object -Property SourceId -AsHashTable

$ptats = $binds["ProgramTermApplnTimeline"] | Group-Object -Property CompositeId -AsHashTable

$binds["Opportunity"] = [System.Collections.Generic.List[PSCustomObject]]::new()
$binds["IndividualApplication"] = [System.Collections.Generic.List[PSCustomObject]]::new()

$preAppStatuses = @('', 'New Inquiry', 'Attempted Contact', 'Contacted')
$invalidDispositions = @('2021', '2022', '2023', '2024', '2025', '2026', '2027', '2028', '2029', '2030')


$fields = @(
    "Id", 
    "Name", 
    "ADEnrollId__c", 
    "Campus__c", 
    "Candidate_Decision__c", 
    "Consultation_Status__c", 
    "Contingency_1__c", 
    "Contingency_2__c", 
    "Contingency_3__c", 
    "Contingency_4__c", 
    "Country_of_Citizenship__c", 
    "CVUE_status_field__c", 
    "Date_Admit__c", 
    "Date_Complete_App__c", 
    "Date_Deposit__c", 
    "Date_Started_App__c", 
    "Date_Submitted_App__c", 
    "Date_Consultation_Completed__c", 
    "Deposit_Deadline__c", 
    "EnrollmentrxRx__Admissions_Status__c", 
    "EnrollmentrxRx__Applicant__c", 
    "EnrollmentrxRx__Applicant__r.First_Source_Code__r.Sparkroom_ID__c", 
    "EnrollmentrxRx__Applicant__r.FirstName", 
    "EnrollmentrxRx__Applicant__r.LastName", 
    "EnrollmentrxRx__Closed_File_Disposition__c", 
    "Term__c", 
    "EnrollmentrxRx__Do_You_Plan_to_Apply_For_Financial_Aid__c", 
    "EnrollmentrxRx__Highest_Level_of_Education__c", 
    "EnrollmentrxRx__How_Did_You_Hear_About_Us__c", 
    "EnrollmentrxRx__Program_of_Interest__c", 
    "EnrollmentrxRx__Program_of_Interest__r.EnrollmentrxRx__Program_Catalog__c", 
    "EnrollmentrxRx__Status_Modified_Date__c", 
    "Estimated_Transfer_Credits__c", 
    "Financial_Aid_Advisor__c", 
    "First_Contact__c", 
    "First_Generation__c", 
    "How_did_you_hear_about_us__c", 
    "Inquiry_Source__c", 
    "Last_Contact__c", 
    "Packaging_Status__c", 
    "Previously_Applied__c", 
    "Program_Catalog__c", 
    "Program_Offered__c", 
    "Program_Offered__r.EnrollmentrxRx__Program_Catalog__c", 
    "Program_Term__c", 
    "Program_Term__r.Program_Offered__c", 
    "Program_Term__r.Program_Offered__r.EnrollmentrxRx__Program_Catalog__c", 
    "Program_Term__r.Term__c", 
    "Program_Term__r.Term__r.EnrollmentrxRx__Term_Start_Date__c", 
    "School__c", 
    "School__r.Name", 
    "Seeking_Transfer_Credit__c", 
    "School_Decision__c", 
    "Student_issued_I_20__c",
    "Student_issued_visa__c", 
    "US_arrival_date__c", 
    "Visa_Type__c", 
    "CreatedDate", 
    "LastModifiedDate", 
    "OwnerId"
)

$predicate = "(Program_Term__r.Term__r.EnrollmentrxRx__Term_Start_Date__c >= 2021-05-10 AND Program_Term__r.Term__r.Associated_Institution__r.Name IN ('$institution')) OR (CreatedDate >= 2022-06-01T00:00:00z AND Program_Term__c = NULL AND School__r.Name IN ('$institution'))"

$enrollmentOpportunities = sf data query -q "SELECT $($fields -join ",") FROM EnrollmentrxRx__Enrollment_Opportunity__c WHERE $predicate" -o $source --json |
Get-Result |
Select-Object -ExpandProperty Records |
ForEach-Object {
    
    $binds["Opportunity"].Add([PSCustomObject]@{
            SourceId = $_.Id
        })
            
   
    $binds["IndividualApplication"].Add([PSCustomObject]@{
            SourceId = $_.Id
        })

    if (-not $_.EnrollmentrxRx__Applicant__c) {
        "[$(Get-Date)]$($_.Id) does not have an associated applicant." | Out-File -FilePath logs\EnrollmentOpportunity.log -Append
        return
    }
            
    $person = $personAccounts[$_.EnrollmentrxRx__Applicant__c]
    if ($null -eq $person) {
        "[$(Get-Date)]$($_.EnrollmentrxRx__Applicant__c) not found in PersonAccount bindings." | Out-File -FilePath logs\EnrollmentOpportunity.log -Append
        return
    }

    $program = if ( $_.Program_Term__r.Program_Offered__r.EnrollmentrxRx__Program_Catalog__c) { $programs[$_.Program_Term__r.Program_Offered__r.EnrollmentrxRx__Program_Catalog__c] } else { $null }
    $term = if ($_.Program_Term__c) { $terms[$_.Program_Term__c] } else { $null } # Potentially add a default term if one wasn't assigned to the source record
    $location = if ($_.Campus__c) { $locations[$_.Campus__c] } else { $null }

    if (-not $program) {
        "[$(Get-Date)]$($_.Program_Term__r.Program_Offered__r.EnrollmentrxRx__Program_Catalog__c) not found in LearningProgram bindings." | Out-File -FilePath logs\EnrollmentOpportunity.log -Append
    }

    $owner = if ($users.containsKey($_.OwnerId)) { $users[$_.OwnerId].TargetId } else { $defaultOwnerId }
    $key = "$($_.Program_Term__r.Program_Offered__r.EnrollmentrxRx__Program_Catalog__c)-$($_.Program_Term__r.Term__c)-$($_.Campus__c)"

    $result = @{
        Opportunity           = $null
        IndividualApplication = $null
    }
      
    $result.opportunity = [PSCustomObject]@{
        Legacy_Id__c             = $_.Id
        Name                     = "$($person.FirstName) $($person.LastName) - $($program.Name)"
        "Account.Legacy_Id__c"   = $_.EnrollmentrxRx__Applicant__c
        CloseDate                = Format-DateTime -d $_.CreatedDate
        StageName                = Get-DictionaryEntry -key "statuses" -value $_.EnrollmentrxRx__Admissions_Status__c
        LeadSource               = $_.Inquiry_Source__c
        Date_First_Contacted__c  = $_.First_Contact__c
        Last_Date_Contacted__c   = $_.Last_Contact__c
        Consultation_DateTime__c = Format-DateTime -d $_.Date_Consultation_Completed__c
        Learning_Program__c      = $program.Id
        Academic_Term__c         = $term.Id
        Campus__c                = $location.TargetId
        Close_Lost_Reason__c     = if ($_.EnrollmentrxRx__Admissions_Status__c -eq 'Closed' -and $_.EnrollmentrxRx__Closed_File_Disposition__c -notin $invalidDispositions) { $_.EnrollmentrxRx__Closed_File_Disposition__c } else { $null }
        Consultation_Status__c   = Get-DictionaryEntry -key "consultationStatus" -value $_.Consultation_Status__c
        Application_Status__c    = if ($_.EnrollmentrxRx__Admissions_Status__c -in @('File Closed', 'Future Start Deposited')) { $null } else { Get-DictionaryEntry -key "appStatuses" -value $_.EnrollmentrxRx__Admissions_Status__c }
        #"Source_Code__r.Lead_Source_ID__c" = $_.ENROLLMENTRXRX__APPLICANT__R.FIRST_SOURCE_CODE__R.SPARKROOM_ID__C ### Commented out while Source Codes do not exist in sandbox
        OwnerId                  = $owner
        CreatedDate              = Format-DateTime -d $_.CreatedDate
        LastModifiedDate         = Format-DateTime -d $_.LastModifiedDate
    }

    # Don't create an IndividualApplication record if the Admissions Status is in one of the pre-application phases
    if (-not $_.EnrollmentrxRx__Admissions_Status__c -or $_.EnrollmentrxRx__Admissions_Status__c -in $preAppStatuses) {
        return $result
    }
    
    
    $schoolDecision = Get-DictionaryEntry -key "schoolDecision" -value $_.School_Decision__c
    if ($schoolDecsion -eq "Admit" -and ($null -ne $contingency -and $contingency.Length -gt 0)) {
        $schoolDecision = "Conditional Admit"
    }
    elseif ($null -eq $schoolDecision -and $_.EnrollmentrxRx__Admissions_Status__c -eq 'Application Accepted') {
        $schoolDecision = $($null -eq $contingency -or $contingency.Length -eq 0) ? $("Admit") : $("Conditional Admit")
    }

    $createdDate = if ($_.Date_Started_App__c) { $_.Date_Started_App__c } else { $_.CreatedDate }
        
    $result.IndividualApplication = [PSCustomObject]@{
        Legacy_Id__c                   = $_.Id
        AccountId                      = $person.Id
        ContactId                      = $person.PersonContactId
        "Opportunity__r.Legacy_Id__c"  = $_.Id
        "Opportunity.Legacy_Id__c"     = $_.Id
        ProgramTermApplnTimelineId     = $ptats[$key].TargetId
        ADEnrollID__c                  = $_.ADEnrollId__c
        Status                         = Get-DictionaryEntry -key "appStatuses" -value $_.EnrollmentrxRx__Admissions_Status__c
        PaymentDate                    = Format-DateTime -d $_.Date_Deposit__c
        CreatedDate                    = Format-DateTime -d $createdDate
        AppliedDate                    = Format-DateTime -d $_.Date_Submitted_App__c
        Date_Application_Reviewable__c = Format-DateTime -d $_.Date_Complete_App__c
        Date_Future_Start_Deposited__c = Format-DateTime -d $_.Date_Deposit__c
        Deposit_Due_Date__c            = Format-DateTime -d $_.Deposit_Deadline__c
        Application_Submitted_Date__c  = Format-DateTime -d $_.Date_Submitted_App__c
        Applying_for_Financial_Aid__c  = $_.Do_You_Plan_to_Apply_For_Financial_Aid__c
        How_did_you_hear_about_us__c   = $_.How_did_you_hear_about_us__c
        Status_Change_Date__c          = Format-DateTime -d $_.EnrollmentrxRx__Status_Modified_Date__c
        Estimated_Transfer_Credits__c  = $_.Estimated_Transfer_Credits__c
        Financial_Aid_Advisor__c       = $_.Financial_Aid_Advisor__c
        Admit_Contingencies__c         = $_.Contingency_1__c
        Admit_Contingencies_2__c       = $_.Contingency_2__c
        Admit_Contingencies_3__c       = $_.Contingency_3__c
        Admit_Contingencies_4__c       = $_.Contingency_4__c
        First_Generation_Student__c    = Get-DictionaryEntry -key "firstGeneration" -value $_.First_Generation__c
        Packaging_Status__c            = $_.Packaging_Status__c
        Has_Previously_Applied__c      = $_.Previously_Applied__c
        Seeking_Transfer_Credit__c     = $_.Seeking_Transfer_Credit__c
        Decision__c                    = Get-DictionaryEntry -key "schoolDecision" -value $_.School_Decision__c
        Applicant_Decision__c          = if ($_.EnrollmentrxRx__Admissions_Status__c -in @('Active', 'Future Start Deposited')) { "Accept" } elseif ($_.EnrollmentrxRx__Admissions_Status__c -eq 'File Closed') { "Decline" } else { $null }
        Category                       = 'Education'
        Active__c                      = $now -le $_.Program_Term__r.Term__r.EnrollmentrxRx__Term_Start_Date__c -or $null -eq $_.Program_Term__r.Term__r.EnrollmentrxRx__Term_Start_Date__c
        RecordTypeId                   = '012Hn000001BLeZIAW'
        LastModifiedDate               = Format-DateTime -d $_.LastModifiedDate
        OwnerId                        = $owner
    }

    return $result
}

$enrollmentOpportunities | Select-Object -ExpandProperty Opportunity | Sort-Object -Property Learning_Program__c | Export-Csv -Path data\Opportunity.csv -NoTypeInformation -Encoding UTF8
$enrollmentOpportunities | Select-Object -ExpandProperty IndividualApplication | Sort-Object -Property AccountId | Export-Csv -Path data\IndividualApplication.csv -NoTypeInformation -Encoding UTF8

Remove-Variable -Name enrollmentOpportunities, personAccounts, programs, terms, locations, ptats

In [ ]:
if ($validateOnly) {
    Write-Host "Validation only. No records will be inserted."
    return
}

sf data upsert bulk --sobject Opportunity --file data\Opportunity.csv --external-id Legacy_Id__c --target-org $target --json --wait 15 | Out-File -FilePath results\Opportunity.json
sf data upsert bulk --sobject IndividualApplication --file data\IndividualApplication.csv --external-id Legacy_Id__c --target-org $target --json --wait 15 | Out-File -FilePath results\IndividualApplication.json

# Update the binds with the target IDs in the event of a successful upsert.
Update-Binds -key "Opportunity" -filePath results\Opportunity.json
Update-Binds -key "IndividualApplication" -filePath results\IndividualApplication.json

In [ ]:
$binds["Task"] = [System.Collections.Generic.List[PSCustomObject]]::new()
$binds["Event"] = [System.Collections.Generic.List[PSCustomObject]]::new()
$binds["Scholarship_Award__c"] = [System.Collections.Generic.List[PSCustomObject]]::new()

$opportunities = Import-Csv -Path data\Opportunity.csv | Group-Object -Property Legacy_Id__c -AsHashTable
$accounts = $binds["PersonAccount"] | Group-Object -Property SourceId -AsHashTable

# Splits the Opportunity IDs into chunks of 750 for the next set of queries
# Avoids exceeding the SOQL query string limit when filtering on a large amount of IDs.
New-ArraySlices -array ($binds["Opportunity"] | Where-Object { $_.TargetId -ne $null }) -size 750 | ForEach-Object {
    $group = $_ | Group-Object -Property SourceId -AsHashTable
    $sourceIds = "'$($group.Keys -join "','")'"

    $fields = @(
        "Id",
        "AccountId",
        "Activity_Type__c",
        "ActivityDate",
        "CallDisposition",
        "CallType",
        "Conversation_Transcript__c",
        "Description",
        "Status",
        "Status_Reason__c",
        "Subject",
        "TaskSubtype",
        "Transfer_Call__c",
        "WhatId",
        "WhoId",
        "CreatedDate",
        "LastModifiedDate"
    )

    sf data query -q "SELECT $($fields -join ",") FROM Task WHERE WhatId IN ($sourceIds)" -o $source --json |
    Get-Result |
    Select-Object -ExpandProperty Records |
    ForEach-Object {
        $binds["Task"].Add([PSCustomObject]@{
                SourceId = $_.Id
            })

        $contact = $accounts[$_.WhoId]

        if (-not $contact) {
            $accountId = $opportunities[$_.WhatId]."Account.Legacy_Id__c"
            if ($accountId) {
                $contact = $accounts[$accountId]
            }
        }

        [PSCustomObject]@{
            Type             = $_.Activity_Type__c
            ActivityDate     = Format-DateTime -d $_.ActivityDate
            CallDisposition  = $_.CallDisposition
            CallType         = $_.CallType
            Transcription__c = $_.Conversation_Transcript__c
            Subject          = $_.Subject
            Description      = $_.Description
            Status           = $_.Status
            Status_Reason__c = $_.Status_Reason__c
            TaskSubtype      = $_.TaskSubtype
            Transfer_Call__c = $_.Transfer_Call__c
            WhatId           = $group[$_.WhatId].TargetId
            WhoId            = $contact.PersonContactId
            CreatedDate      = Format-DateTime -d $_.CreatedDate
            LastModifiedDate = Format-DateTime -d $_.LastModifiedDate
        }
    } | Export-Csv -Path data\Task.csv -NoTypeInformation -Encoding UTF8 -Append
    
    $fields = @(
        "Id",
        "AccountId",
        "ActivityDate",
        "ActivityDateTime",
        "Appointment_Status__c",
        "Conversation_Transcript__c",
        "Description",
        "EndDate",
        "EndDateTime",
        "EventSubtype",
        "IsAllDayEvent",
        "Location",
        "ShowAs",
        "StartDateTime",
        "Subject",
        "Type",
        "WhatId",
        "WhoId",
        "CreatedDate",
        "LastModifiedDate"
    )
    
    sf data query -q "SELECT $($fields.Keys -join ",") FROM Event WHERE WhatId IN ($sourceIds)" -o $source --json |
    Get-Result |
    Select-Object -ExpandProperty Records |
    ForEach-Object {

        $binds["Event"].Add([PSCustomObject]@{
                SourceId = $_.Id
            })

        $contact = $accounts[$_.WhoId]

        if (-not $contact) {
            $accountId = $opportunities[$_.WhatId]."Account.Legacy_Id__c"
            if ($accountId) {
                $contact = $accounts[$accountId]
            }
        }

        [PSCustomObject]@{
            ActivityDate          = Format-DateTime -d $_.ActivityDate
            ActivityDateTime      = Format-DateTime -d $_.ActivityDateTime
            Appointment_Status__c = $_.Appointment_Status__c
            Transcription__c      = $_.Conversation_Transcript__c
            Description           = $_.Description
            EndDateTime           = Format-DateTime -d $_.EndDateTime
            EventSubtype          = $_.EventSubtype
            IsAllDayEvent         = $_.IsAllDayEvent
            Location              = $_.Location
            ShowAs                = $_.ShowAs
            StartDateTime         = Format-DateTime -d $_.StartDateTime
            Subject               = $_.Subject
            Type                  = $_.Type
            WhatId                = $group[$_.WhatId].TargetId
            WhoId                 = $contact.PersonContactId
            CreatedDate           = Format-DateTime -d $_.CreatedDate
            LastModifiedDate      = Format-DateTime -d $_.LastModifiedDate
        }

    } | Export-Csv -Path data\Event.csv -NoTypeInformation -Encoding UTF8 -Append

    $fields = @(
        "Id",
        "AllowedAwardAmounts__c",
        "Application__c",
        "Award_Date__c",
        "Award_Status__c",
        "AwardCriteria1Options__c",
        "AwardCriteria2Options__c",
        "DateAccepted__c",
        "DateDeclined__c",
        "ResponseDueDate__c",
        "Scholarship_Location__c",
        "CreatedDate",
        "LastModifiedDate"
    )
    
    sf data query -q "SELECT $($fields.Keys -join ",") FROM Scholarship_Award__c WHERE Application__c IN ($sourceIds)" -o $source --json |
    Get-Result |
    Select-Object -ExpandProperty Records |
    ForEach-Object {

        $binds["Scholarship_Award__c"].Add([PSCustomObject]@{
                SourceId = $_.Id
            })

        [PSCustomObject]@{
            "Application__r.Legacy_Id__c" = $_.Application__c
            "Scholarship__r.Legacy_Id__c" = $_.Scholarship_Location__c
            Award_Amount__c               = $_.AllowedAwardAmounts__c
            Award_Date__c                 = Format-DateTime -d $_.Award_Date__c
            Award_Status__c               = $_.Award_Status__c
            Award_Criteria_1_Options__c   = $_.AwardCriteria1Options__c
            Award_Criteria_2_Options__c   = $_.AwardCriteria2Options__c
            Date_Accepted__c              = Format-DateTime -d $_.DateAccepted__c
            Date_Declined__c              = Format-DateTime -d $_.DateDeclined__c
            Response_Due_Date__c          = Format-DateTime -d $_.ResponseDueDate__c
            CreatedDate                   = Format-DateTime -d $_.CreatedDate
            LastModifiedDate              = Format-DateTime -d $_.LastModifiedDate
        }
    } | Export-Csv -Path data\Scholarship_Award__c.csv -NoTypeInformation -Encoding UTF8 -Append
}

In [ ]:
if ($validateOnly) {
    Write-Host "Validation only. No records will be inserted."
    return
}

sf data upsert bulk --sobject Task --file data\Task.csv --external-id Legacy_Id__c --target-org $target --json --wait 10 | Out-File -FilePath results\Task.json
sf data upsert bulk --sobject Event --file data\Event.csv --external-id Legacy_Id__c --target-org $target --json --wait 10 | Out-File -FilePath results\Event.json
sf data upsert bulk --sobject Scholarship_Award__c --file data\Scholarship_Award__c.csv --external-id Id --target-org $target --json --wait 10 | Out-File -FilePath results\Scholarship_Award__c.json

# Update the binds with the target IDs in the event of a successful upsert.
Update-Binds -key "Task" -filePath results\Task.json
Update-Binds -key "Event" -filePath results\Event.json
Update-Binds -key "Scholarship_Award__c" -filePath results\Scholarship_Award__c.json